In [2]:
# huggingface-cli login

In [7]:
# =======================
# Setup (一度だけ実行)
# =======================
# pip install -U "transformers>=4.41" "sentence-transformers>=3.0.0" "huggingface_hub>=0.23" torch

from typing import List
import numpy as np
from numpy.linalg import norm

from huggingface_hub import login
from sentence_transformers import SentenceTransformer

# =======================
# ② EmbeddingGemma (検索/類似度) の比較
# =======================
EMB_MODEL_ID = "google/embeddinggemma-300m"  # 埋め込みモデル
emb_model = SentenceTransformer(EMB_MODEL_ID, use_auth_token=True)

def rank_by_similarity(query: str, docs: List[str]):
    q = emb_model.encode_query(query)              # shape: (768,)
    d = emb_model.encode_document(docs)            # shape: (N, 768)
    # SentenceTransformersのsimilarityでもOKだが、ここではcos類似度で明示
    sims = (q @ d.T) / (norm(q) * norm(d, axis=1)) # shape: (N,)
    order = np.argsort(-sims)                      # 降順ソート
    return sims, order



In [9]:
query = "赤い惑星として知られている惑星はどれですか？"
documents = [
    "金星は地球の双子と呼ばれることがあります。",
    "火星は赤っぽい外観で知られ、赤い惑星と呼ばれています。",
    "木星は太陽系で最大の惑星で、大赤斑があります。",
    "土星はその美しい環で有名ですが、赤い惑星と間違われることもあります。"
]

print("=== EmbeddingGemma（検索/類似度） ===")
sims, order = rank_by_similarity(query, documents)
for idx in order:
    print(f"{sims[idx]:.3f} | {documents[idx]}")

# 期待挙動：
# 検索：類似度が最も高い文として「火星...赤い惑星...」が上位に来る

=== EmbeddingGemma（検索/類似度） ===
0.623 | 火星は赤っぽい外観で知られ、赤い惑星と呼ばれています。
0.609 | 土星はその美しい環で有名ですが、赤い惑星と間違われることもあります。
0.547 | 木星は太陽系で最大の惑星で、大赤斑があります。
0.491 | 金星は地球の双子と呼ばれることがあります。
